#Artifact Detection: Bubbles



import relevant libraries:

In [22]:
import cv2
import numpy as np
import glob

###Helper functions for bubble_detection:

1) Filter contours:

In [9]:
def filter_by_size(contours, low_thresh = 100, up_thres = 1000):
    '''Helper function for bubble_detection. Takes in a list of contour coordinates. 
    Filters out contours that are above or below a given threshold
    Returns a filtered list of contours.'''
    filtered = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < up_thres and area > low_thresh:
            filtered.append(c)
    return filtered
            

def filter_by_ratio(contours):
    '''Helper function for bubble_detection. Takes in a list of contour coordinates. 
    Filters out contours by aspect ratio. 
    Settings were determined by trial and error. 
    Returns a filtered list of contours.'''
    filtered = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        aspect_ratio = float(w)/h
        
        if aspect_ratio > .5 and aspect_ratio < 1:   
            filtered.append(c)
            
    return filtered

def filter_contours(contours):
    '''Takes a list of potential contours as an argument.
    Calls two filter functions, filter_by_size and filter_by_ratio, to filter contours.
    Returns a filtered list of contours.'''
    f_contours1 = filter_by_size(contours)
    f_controus2 = filter_by_ratio(f_contours1)
    return f_controus2
        

2) Basic image processing:

In [10]:
def fuss_with_image(img):
    
    '''Helper function for test_image. 
    Makes the image a little smaller, applies a blur to improve contour detection.
    '''    
    #make the image a little smaller for my computer (macbook air)
    r = 500.0 / img.shape[1]
    dim = (500, int(img.shape[0] * r))
    # perform the actual resizing of the image and show it
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    img_blur = cv2.bilateralFilter(resized,9,75,75)
    
    return img_blur

##Main Function: bubble_detector

In [11]:
def bubble_detector(filename):
    '''Takes in a file as an argument. Reads the file, detects bubble-like contours,
    and returns two arguments:
    1) an image with the contours overlayed on the original image file
    2) A tuple of if bubbles were found, and the number of bubbles detected.
    '''
    #create a default tuple, which assumes that no bubbles are detected
    outcome = (False, 0)
    
    #read the file
    img = cv2.imread(filename,0)
    #call fuss_with_image to pre-process the image a small amount
    image = fuss_with_image(img)
    
    #create the countours around identified bubbles 
    ret,thresh = cv2.threshold(image,127,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #filter the list of contours to match actual bubbles
    c = filter_contours(contours)
    
    #if any contours still exist after filtering them, change the outcome tuple:
    if len(c) > 0:
        outcome = (True, len(c))

    #overlay the contours onimage
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    cv2.drawContours(image,c,-1,(0,255,0),3)
    
    #return an image file with contours drawn on, and the outcome (were bubbles found?)
    return image, outcome

#Run bubble_detector:

## 1) Run bubble_detector on the provided images:

In [26]:
def process_folder_for_images(image_list):
    '''Processes all image files by running through through the bubble_dector function.
    Run bubble detector by iterating over a list of files on the computer, and calling bubble_detector on every file.
    Put the image file returned by bubble_dector into folder named 'output' '''

    for scope_file in image_list[1:]:
        #run every file through the function, bubble_detector

        image, outcome = bubble_detector(scope_file)

        #after each file is being processed, print out the filename and outcome
        print scope_file, ' Bubbles: ',outcome[1]

        #write a new image with bubbles traced to an output folder
        output_name = './output/'+scope_file[14:20]+'_bubble_count_'+str(outcome[1])+'.png'
        cv2.imwrite(output_name, image)
        
image_list = glob.glob("./data/images/*.png")

process_folder_for_images(image_list)

./data/images/15032_e3587904-27f8-43c5-bdc5-cb345ace7c34.png  Bubbles:  0
./data/images/15042_0c45140a-d88c-4b14-85b9-234e990721b1.png  Bubbles:  1
./data/images/15052_27df81d9-975f-4126-8084-f35554e8468c.png  Bubbles:  1
./data/images/15062_e268b731-1422-4269-b0c4-d2f1ab73d9fd.png  Bubbles:  1
./data/images/15072_cb9b597d-ee32-4e9b-8865-5f0a2b81a702.png  Bubbles:  3
./data/images/15082_365e215d-1afd-4407-99ba-54dd70172b96.png  Bubbles:  3
./data/images/15092_6242149e-5f21-4c55-b32a-c1f4e8263d72.png  Bubbles:  1
./data/images/15102_4eef5851-08cf-4836-9027-66945996b224.png  Bubbles:  1
./data/images/15112_2916139e-470c-470f-ad4d-936ee99efda3.png  Bubbles:  1
./data/images/15122_25a62679-8c64-4fd8-a06b-9bb5f143fc43.png  Bubbles:  1
./data/images/15132_ce241f16-7155-4903-80a5-2e4297408eb8.png  Bubbles:  0
./data/images/15142_9915a3d9-d958-4431-9322-d37ab6f42a88.png  Bubbles:  0
./data/images/15152_60a860de-fcc6-4d12-b82f-f09e00c3c46d.png  Bubbles:  1
./data/images/15162_71a7b804-03a3-462d

## 2) Run bubble_detector on one image:

In [15]:
def process_one_image(filepath, output_name):
    #call function bubble_detector
    image, outcome = bubble_detector(filepath)
    
    #print result, and write output to a new file
    print ' Bubbles: ',outcome[1]
    cv2.imwrite(output_name, image)
    
filepath = 'data/images/15172_08e4729c-2887-4eea-9695-85882837f1bd.png'
process_one_image(filepath, 'output/test.png')

  Bubbles:  2



#Example Code to View Metadata

In [6]:
import json
filename = 'data/metadata.json'
with open(filename, 'r') as f:
    l = f.read()
data = json.loads(l)

The metadata is now in a python dictionary, and can be easily looked in potential future analysis not done in this notebook. 

In [7]:
print 'Number of Images:', len(data), '\n'  
print data[0].keys()  #the information associated with each image

Number of Images: 56 

[u'stats', u'tags', u'drivers', u'time', u'offset', u'sliceIndex', u'kesmName', u'_id', u'properties', u'imageType']


In [8]:
#example: Get a list of all the ids
print [picturedict['_id'] for picturedict in data]    

[u'14bcf5f8-d543-4c41-bb28-436e0f8c68ed', u'e3587904-27f8-43c5-bdc5-cb345ace7c34', u'0c45140a-d88c-4b14-85b9-234e990721b1', u'27df81d9-975f-4126-8084-f35554e8468c', u'e268b731-1422-4269-b0c4-d2f1ab73d9fd', u'cb9b597d-ee32-4e9b-8865-5f0a2b81a702', u'365e215d-1afd-4407-99ba-54dd70172b96', u'6242149e-5f21-4c55-b32a-c1f4e8263d72', u'4eef5851-08cf-4836-9027-66945996b224', u'2916139e-470c-470f-ad4d-936ee99efda3', u'25a62679-8c64-4fd8-a06b-9bb5f143fc43', u'ce241f16-7155-4903-80a5-2e4297408eb8', u'9915a3d9-d958-4431-9322-d37ab6f42a88', u'60a860de-fcc6-4d12-b82f-f09e00c3c46d', u'71a7b804-03a3-462d-a587-cbe4f618b794', u'08e4729c-2887-4eea-9695-85882837f1bd', u'828c7875-a9aa-4f8e-8e50-9e792e0966f1', u'86795e89-737e-46d6-823b-6407c7b3ad74', u'4f959e0a-681a-48c5-9316-e9851d6c8a76', u'f1409bc7-cc6a-4aad-a8c2-9867529f8ec5', u'8391a821-6309-4aae-abdb-481899604656', u'79460bb1-becc-4168-9461-65587b66203b', u'4a8d1e87-eed9-4cc7-90fe-84777241bbcf', u'7bd7c7ed-6cfb-49d9-b0f8-90a6f39678de', u'83c0c7c9-3cb8